In [1]:
%matplotlib inline
import dcgan
import tensorflow as tf
import os
import ipywidgets as widgets
from matplotlib import pyplot as plt

# size of latent z vector
nz = widgets.IntText(
    value=512,
    description='nz'
)

ngf =  widgets.IntText(
    value=32,
    description='ngf'
)

ndf =  widgets.IntText(
    value=16,
    description='ndf'
)

n_epoch = widgets.IntText(
    value=25,
    description='n epoch'
)

lr = widgets.FloatText(
    value = 0.002,
    description = 'learning rate'
)

beta1 = widgets.FloatText(
    value = 0.5,
    description = 'beta 1'
)

im_size = widgets.IntText(
    value = 64,
    description = 'im size'
)

file_path = widgets.Text(
    description = 'path'
)

u_box = widgets.HBox([nz, ngf, ndf, n_epoch])
b_box = widgets.HBox([lr, beta1,im_size])
v_b = widgets.VBox([u_box,b_box,file_path])

display(v_b)

In [2]:
import h5py

tf.enable_eager_execution()

BUFFER_SIZE = 60000
BATCH_SIZE = 10

f = h5py.File(file_path.value, 'r')

train_dataset = tf.data.Dataset.from_tensor_slices(f['data']).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [3]:
nc = 1

netG = dcgan.make_generator_model(im_size.value, nz.value, nc, ngf.value)
netD = dcgan.make_discriminator_model(im_size.value, nc, ndf.value)

TypeError: make_generator_model() missing 4 required positional arguments: 'isize', 'nz', 'nc', and 'ngf'

In [ ]:
d_optimizer = tf.keras.optimizers.Adam(lr.value, beta_1=beta1.value)
g_optimizer = tf.keras.optimizers.Adam(lr.value, beta_1=beta1.value)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.metrics.binary_crossentropy(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.metrics.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return tf.keras.metrics.binary_crossentropy(tf.ones_like(fake_output), fake_output)

In [ ]:
import time

for epoch in range(n_epoch.value):
    start = time.time()
    for image_batch in train_dataset:       
        noise = tf.random.normal([BATCH_SIZE,1,1,1,nz.value])
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = netG(noise, training=True)
            
            fake_output = netD(generated_images, training=True)
            
            real_output = netD(image_batch, training=True)
            gen_loss = generator_loss(fake_output)
            #disc_loss = discriminator_loss(real_output, fake_output)

            #gradients_of_generator = gen_tape.gradient(gen_loss, netG.trainable_variables)
            #gradients_of_discriminator = disc_tape.gradient(disc_loss, netD.trainable_variables)

            #g_optimizer.apply_gradients(zip(gradients_of_generator, netG.trainable_variables))
            #d_optimizer.apply_gradients(zip(gradients_of_discriminator, netD.trainable_variables))

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
